In [1]:
import requests
import json
import ast
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic U0VBX############################=='#got the apikey using postman--->convert to python method
}

In [8]:
#Get activitype,campaign name and asset id to filter the required results, create the sync on bulk and return json file
def api(actype,campname,aid): 
  url="https://secure.p03.eloqua.com/api/bulk/2.0/activities/exports"
  payload={
  "filter": "'{{Activity.Type}}'= '"+str(actype)+"' AND '{{Activity.Campaign.Field(CampaignName)}}'= '"+str(campname)+"' AND '{{Activity.Asset.Id}}'='"+str(aid)+"'",
  "name": "api export",
  "fields": {#defining the required fields. Please refer the official documentation for additional fields
    "ActivityType": "{{Activity.Type}}",
    "CampaignName": "{{Activity.Campaign.Field(CampaignName)}}",
    "ContactId": "{{Activity.Contact.Id}}",
    "AssetId": "{{Activity.Asset.Id}}",
    "EmailAddress": "{{Activity.Field(EmailAddress)}}"
    }
  }

  response = requests.request("POST" , url, headers=headers, data=json.dumps(payload))
  dict1= ast.literal_eval(response.text)
  sync1=dict1["uri"] #we get sync number in the response and will be required in post req
  url = "https://secure.p03.eloqua.com/API/bulk/2.0/syncs"
  payload = json.dumps({
  "syncedInstanceUri": sync1
  })
  response = requests.request("POST", url, headers=headers, data=payload)
  dict2= ast.literal_eval(response.text)
  sync2=dict2["uri"] #This is the final sync url which will be used to fetched the results
  while ('"status":"success"'not in response.text): #waiting for the sync to complete then load the next step
    url = f"https://secure.p03.eloqua.com/API/bulk/2.0{sync2}"
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)
  url = f"https://secure.p03.eloqua.com/API/bulk/2.0{sync2}/data?limit=50000" #we can use offsets in case you want additional items
  payload = {}
  response = requests.request("GET", url, headers=headers, data=payload)
  #return(response.json()['totalResults'])
  return(response.json())#returning the entire json file

In [6]:
#Getting all email groups belonging to the campaign
def assets(cname):
    url=f"https://secure.p03.eloqua.com/api/rest/2.0/assets/campaigns?depth=complete&search=name='{cname}'"
    payload={}
    response = requests.request("GET" , url, headers=headers, data=payload)
    data=response.json()['elements'][0]['elements']
    assets=[]
    for d in data:
        if d['type']=='CampaignEmail':
            if d['emailId'] not in assets:
                assets.append(d['emailId'])
    return(assets)

In [16]:
# Fetching the asset names from the provided asset id
def name(id):
    url=f"https://secure.p03.eloqua.com/api/REST/2.0/assets/email/{id}"
    payload={}
    response = requests.request("GET" , url, headers=headers, data=payload)
    return response.json()


In [14]:
acttype=['EmailSend','EmailOpen','EmailClickthrough','Bounceback'] #define activity fields you want
datajson=[] 
camp='<your campaign name>' #create an array in case of multiple campaigns and iterate entire logic below

for i in assets(camp):
    Send=api(acttype[0],camp,i)
    Open= api(acttype[1],camp,i)
    Click=api(acttype[2],camp,i)
    Bounce=api(acttype[3],camp,i)
    datajson.append([i,Send['totalResults'],Open['totalResults'],Click['totalResults'],Bounce['totalResults']])


In [17]:
#creating the data in format we require to print in the output file
data=[]
for i in datajson:
    a=name(i[0])
    data.append([camp,a['name'],a['subject'],i[1],i[2],i[3],i[4]])

In [18]:
#Feed the data into csv files
import csv
filename = 'output_file.csv'
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Campaign', 'EmailSegment', 'SubjectLine','Total EmailSends','Total EmailOpens','Total EmailClickthrough','Total Bounces'])
    writer.writerows(data)